In [1]:
import os

In [2]:
%pwd

'd:\\chest-cancer-classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\chest-cancer-classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [6]:
from chest_cancer.constants import *
from chest_cancer.utils.commen import read_yaml, create_directories     

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [8]:
import os
import zipfile
import gdown
from chest_cancer import logger
from chest_cancer.utils.commen import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"File extracted at: {unzip_path}")

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

2025-01-13 11:56:59,386 - INFO - commen - yaml file: config\config.yaml loaded successfully
2025-01-13 11:56:59,394 - INFO - commen - yaml file: params.yaml loaded successfully
2025-01-13 11:56:59,396 - INFO - commen - Created directory at artifacts
2025-01-13 11:56:59,400 - INFO - commen - Created directory at artifacts/data_ingestion
2025-01-13 11:56:59,406 - INFO - 2536846962 - Downloading data from https://drive.google.com/file/d/1RxBoKtDeEwoBsh9BILgV4NW71GnyLAhO/view?usp=sharing into file artifacts/data_ingestion/data.zip


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1RxBoKtDeEwoBsh9BILgV4NW71GnyLAhO
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1RxBoKtDeEwoBsh9BILgV4NW71GnyLAhO&confirm=t&uuid=8038727b-7f3d-47b8-baa1-9ee0b74be51a
To: d:\chest-cancer-classification\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:09<00:00, 5.06MB/s]

2025-01-13 11:57:15,186 - INFO - 2536846962 - Downloaded data from https://drive.google.com/file/d/1RxBoKtDeEwoBsh9BILgV4NW71GnyLAhO/view?usp=sharing into file artifacts/data_ingestion/data.zip


2025-01-13 11:57:16,239 - INFO - 2536846962 - File extracted at: artifacts/data_ingestion
